In [1]:
# Setup and Imports
!nvidia-smi  # Check GPU availability

import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import cv2
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os

# Seed everything for reproducibility
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

Fri Mar 21 09:05:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
# Data Preparation
train = pd.read_csv("/kaggle/input/lacuna-solar-survey-challenge/Train.csv")

# Create mappers for metadata
placement_mapper = train[["ID", "placement"]].drop_duplicates().set_index("ID").to_dict()
img_origin_mapper = train[["ID", "img_origin"]].drop_duplicates().set_index("ID").to_dict()

# Aggregate counts and add metadata
train_df = train.groupby("ID").sum().reset_index()[["ID", "boil_nbr", "pan_nbr"]]
train_df["img_origin"] = train_df["ID"].map(img_origin_mapper["img_origin"])
train_df["placement"] = train_df["ID"].map(placement_mapper["placement"])
train_df["path"] = "/kaggle/input/lacuna-solar-survey-challenge/images/" + train_df["ID"] + ".jpg"

# Stratified K-Fold for balanced splits
train_df["stratify_label"] = train_df[["boil_nbr", "pan_nbr"]].sum(axis=1)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_df["fold"] = -1
for fold, (_, valid_idx) in enumerate(skf.split(train_df, train_df["stratify_label"])):
    train_df.loc[valid_idx, "fold"] = fold

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [3]:
# Data Transformations
train_transforms = A.Compose([
    A.Resize(768, 768),  # Higher resolution for detail
    A.Rotate(limit=30, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=5, p=0.3),
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.5),  # Targeted dropout
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_transforms = A.Compose([
    A.Resize(768, 768),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [4]:
# Custom Dataset with Metadata
class SolarPanelDataset(Dataset):
    def __init__(self, dataframe, transform=None, to_train=True):
        self.dataframe = dataframe
        self.transform = transform
        self.to_train = to_train

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = cv2.imread(row["path"])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            image = self.transform(image=image)["image"]
            
        origin = 0 if row["img_origin"] == "D" else 1  # D=0, G=1
        placement = 0 if row["placement"] == "roof" else 1  # roof=0, ground=1
        
        if self.to_train:
            target = torch.tensor([row["boil_nbr"], row["pan_nbr"]], dtype=torch.float32)
            
            return image, torch.tensor(origin), torch.tensor(placement), target
            
        return image, torch.tensor(origin), torch.tensor(placement)

In [5]:
# Model Definition with Metadata Fusion
class EfficientNetV2Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model("tf_efficientnet_b5", pretrained=True)
        in_features = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Identity()
        self.origin_embed = nn.Embedding(2, 8)
        self.placement_embed = nn.Embedding(2, 8)
        self.regression_head = nn.Sequential(
            nn.Linear(in_features + 16, 128),  # 16 = 8 (origin) + 8 (placement)
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 2)
        )

    def forward(self, images, origins, placements):
        img_features = self.backbone(images)
        origin_features = self.origin_embed(origins)
        placement_features = self.placement_embed(placements)
        fused = torch.cat([img_features, origin_features, placement_features], dim=1)
        return self.regression_head(fused)

In [6]:
# Training and Validation Function
def train_and_validate(fold, train_loader, valid_loader, num_epochs=50):
    model = EfficientNetV2Regressor().cuda()
    criterion = nn.L1Loss()  # MAE-focused
    optimizer = optim.AdamW(model.parameters(), lr=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    scaler = torch.cuda.amp.GradScaler()
    best_loss = float("inf")
    best_model_path = f"best_model_fold{fold}.pth"

    for epoch in range(num_epochs):
        # Training
        model.train()
        epoch_loss = 0.0
        for images, origins, placements, targets in tqdm(train_loader, desc=f"Fold {fold} Epoch {epoch+1}/{num_epochs} - Training"):
            images, origins, placements, targets = images.cuda(), origins.cuda(), placements.cuda(), targets.cuda()
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = model(images, origins, placements)
                loss = criterion(outputs, targets)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            epoch_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, origins, placements, targets in tqdm(valid_loader, desc=f"Fold {fold} Epoch {epoch+1}/{num_epochs} - Validation"):
                images, origins, placements, targets = images.cuda(), origins.cuda(), placements.cuda(), targets.cuda()
                outputs = model(images, origins, placements)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        val_loss /= len(valid_loader)
        print(f"Fold {fold} Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}")
        
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), best_model_path)
        
        scheduler.step()

    return best_model_path

In [7]:
# TTA Prediction Function
def predict_with_tta(model, loader):
    model.eval()
    preds = []
    with torch.no_grad():
        for images, origins, placements in tqdm(loader, desc="TTA Prediction"):
            images, origins, placements = images.cuda(), origins.cuda(), placements.cuda()
            pred_orig = model(images, origins, placements)
            pred_flip = model(torch.flip(images, dims=[3]), origins, placements)  # Horizontal flip
            pred = (pred_orig + pred_flip) / 2
            preds.append(pred.cpu().numpy())
    return np.concatenate(preds)

In [8]:
# Main Execution: Full Cross-Validation and Test Prediction
num_folds = 1
num_epochs = 30

# Train across all folds
for fold in range(num_folds):
    train_data = train_df[train_df["fold"] != fold].reset_index(drop=True)
    valid_data = train_df[train_df["fold"] == fold].reset_index(drop=True)
    
    dataset_train = SolarPanelDataset(train_data, transform=train_transforms)
    dataset_valid = SolarPanelDataset(valid_data, transform=test_transforms)
    
    train_loader = DataLoader(dataset_train, batch_size=8, shuffle=True, num_workers=os.cpu_count())
    valid_loader = DataLoader(dataset_valid, batch_size=8, shuffle=False)
    
    best_model_path = train_and_validate(fold, train_loader, valid_loader)

model.safetensors:   0%|          | 0.00/122M [00:00<?, ?B/s]

<ipython-input-6-b71f595ecedf>:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Fold 0 Epoch 1/50 - Training:   0%|          | 0/332 [00:00<?, ?it/s]<ipython-input-6-b71f595ecedf>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Fold 0 Epoch 1/50 - Validation: 100%|██████████| 83/83 [02:32<00:00,  1.83s/it]


Fold 0 Epoch 1/50, Train Loss: 2.3571, Val Loss: 2.4979


Fold 0 Epoch 2/50 - Validation: 100%|██████████| 83/83 [02:14<00:00,  1.62s/it]


Fold 0 Epoch 2/50, Train Loss: 2.0419, Val Loss: 1.9602


...

Fold 0 Epoch 48/50, Train Loss: 0.4323, Val Loss: 1.5724


Fold 0 Epoch 49/50 - Validation: 100%|██████████| 83/83 [02:13<00:00,  1.60s/it]


Fold 0 Epoch 49/50, Train Loss: 0.4362, Val Loss: 1.4634


Fold 0 Epoch 50/50 - Validation: 100%|██████████| 83/83 [02:13<00:00,  1.61s/it]

Fold 0 Epoch 50/50, Train Loss: 0.4093, Val Loss: 1.4705


In [9]:
# Test Prediction with Aggregation
test_df = pd.read_csv("/kaggle/input/lacuna-solar-survey-challenge/Test.csv")
test_df["path"] = "/kaggle/input/lacuna-solar-survey-challenge/images/" + test_df["ID"] + ".jpg"
dataset_test = SolarPanelDataset(test_df, transform=test_transforms, to_train=False)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False)

model = EfficientNetV2Regressor().cuda()
test_preds = np.zeros((len(test_df), 2))
for fold in range(num_folds):
    model.load_state_dict(torch.load(f"best_model_fold{fold}.pth"))
    fold_preds = predict_with_tta(model, test_loader)
    test_preds += fold_preds
test_preds /= num_folds

# Create Submission
submission = pd.DataFrame()
submission["ID"] = np.repeat(test_df["ID"].values, 2)
submission["ID"] = submission["ID"] + np.tile(["_boil", "_pan"], len(test_df))
submission["Target"] = test_preds.flatten().clip(0, 1000)  # Clip but no rounding
submission.to_csv("submission.csv", index=False)
print("Submission saved as submission.csv!")

<ipython-input-9-2a9cdbc6386d>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"best_model_fold{fold}.pth"))
TTA Prediction: 100%|███████

Submission saved as submission.csv!


In [10]:
submission["Target"] = test_preds.flatten().clip(0, 1000)  # Clip but no rounding (0, 1000)

In [11]:
# submission.loc[submission['Target'] > 100, 'Target'] *= 1.02

In [12]:
submission["Target"] = submission["Target"].round()

In [13]:
submission.to_csv("submission2.csv", index=False)
print("Submission saved as submission.csv!")

Submission saved as submission.csv!
